# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [ ]:
import numpy as np
import os

#from scipy.misc import imread, imresize
from imageio import  imread
from PIL import Image  

import datetime
import glob

#from tqdm.autonotebook import tqdm
import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

We set the random seed so that the results don't vary drastically.

In [ ]:
#!pip show tensorflow

In [ ]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)
#tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [ ]:
# train_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/train.csv').readlines())
# val_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/val.csv').readlines())

train_doc = np.random.permutation(open('train.csv').readlines()) #List of all training videos
val_doc = np.random.permutation(open('val.csv').readlines())  #List of all validation videos

#for CNN-RNN
batch_size = 100 #experiment with the batch size. Number of videos in each batch

#for CNN
#batch_size = 40 #experiment with the batch size. Number of videos in each batch

imsize=(128,128)

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

##### Resizing every time is consuming resource and time. Therefore it is done at the outset of starting experiments.After few experiments I decided 128x128 is serve our purpose therefore all images will be created with this dimension and preserved. Only these images will be referred during experiments. Name of Resized files starts with "rs_"

In [ ]:
#delete old resized file. Trying with different size of image. 
#If old size does not work then we need to remove those thousands of files
def delete_old_files(folders, path):
    for folder in folders:
        fileList = glob.glob(path +"/"+folder.split(";")[0]+"/rs_*" )
        for f in fileList:
            try:
                os.remove(f)
            except:
                print("Error while deleting file : ", f)
            print ("Deleting ",f)


### Uncomment Below Lines if we want to delete previously created images

In [ ]:
#delete_old_files(train_doc, "train")
#delete_old_files(val_doc, "val")

In [ ]:
#Resize all image and keep them on disk so that for we need not to do this in memory for every epoch
from tqdm.autonotebook import tqdm
def resize_images(source_path):
    folders = os.listdir(source_path)
    for i in tqdm( range(0, len(folders))):
        folder=folders[i]
        imgs = os.listdir(source_path+'/'+ folder) #.split(';')[0]) # read all the images in the folder
        for img in imgs:
            if (img,img[0:3])!="rs_":
                img_file = source_path+'/'+folder +"/"+img
                rs_img_file = source_path+'/'+folder+'/rs_'+img
                image = Image.open(img_file)
                
                left  = int((image.width*0.05)/2)
                right = image.width-2*left
                top   =  int((image.height*0.05)/2)
                bottom = image.height- 2*top
                
                image = image.crop((left, top, right, bottom)) 
                image = image.resize(imsize)
                
                image.save(rs_img_file)
                print("Resizing: ",rs_img_file)
        
# train_path = '/notebooks/storage/Final_data/Collated_training/train'
# val_path = '/notebooks/storage/Final_data/Collated_training/val'

train_path = 'train'
val_path   = 'val'

### Uncomment following lines if you want to create images. Run only one time

In [ ]:
resize_images(train_path)
resize_images(val_path)

In [ ]:
img_idx = np.array( range(0,30,3))
imgs_from_1video=len(img_idx) #Number of Images from each video

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    
    while True:  
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size # calculate the number of batches
        remaining_videos= len(t)%batch_size
 
        for batch in tqdm( range(num_batches) ): # we iterate over the number of batches
            batch_data = np.zeros((batch_size, imgs_from_1video ,imsize[0] , imsize[1],3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in              
                    if imgs[item][0:3]=="rs_":
                        img_file = source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]
                        image = imread(img_file).astype(np.float32)

                        batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                        batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                        batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if remaining_videos!=0:      
            batch_data   = np.zeros((remaining_videos, imgs_from_1video ,imsize[0] , imsize[1],3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((remaining_videos,5)) # batch_labels is the one hot representation of the output
            
            for i in range(remaining_videos):
                    folder= i + batch*batch_size + 1
                    imgs = os.listdir(source_path+'/'+ t[folder].split(';')[0]) # read all the images in the folder

                    for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in

                        if imgs[item][0:3]=="rs_":
                            img_file = source_path+'/'+ t[folder].strip().split(';')[0]+'/'+imgs[item]
                            image = imread(img_file).astype(np.float32)
                            
                            batch_data[i,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                            batch_data[i,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                            batch_data[i,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                            
                    batch_labels[i, int(t[folder].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [ ]:
curr_dt_time = datetime.datetime.now()

num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50# choose the number of epochs
print ('# epochs =', num_epochs)

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

## Commenting Conv3D Model

#### I am not submitting this model because it is too complex and need lots of memory (250 MB). I am not able to upload this modle.

num_classes=5

from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D

from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras import optimizers
from keras.layers import Dropout
from keras.optimizers import Adam, SGD
from keras.regularizers import l2, L1L2

model = Sequential()
model.add(Conv3D(32, (2, 3, 3), padding='same',
                 input_shape=(imgs_from_1video, imsize[0],imsize[1],3)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv3D(32, (2, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))


model.add(Conv3D(64, (2, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv3D(64, (2, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#model.add(Dropout(0.25))


model.add(Conv3D(128, (2, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv3D(128, (2, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.8))

model.add(Dense(num_classes))
model.add(Activation('softmax'))
#write your model here

## CNN + RNN Model

In [ ]:
num_classes=5

from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D

from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras import optimizers
from keras.optimizers import Adam, SGD
from keras.layers import Dropout
from keras.regularizers import l2, L1L2

model = Sequential()

model.add(TimeDistributed(Conv2D(32, (3,3), strides=(2, 2), activation='relu',
                                 padding='same'), input_shape=(imgs_from_1video, imsize[0],imsize[1], 3)))
model.add(BatchNormalization())
model.add(TimeDistributed(Conv2D(32, (3,3), kernel_initializer="he_normal", activation='relu')))
model.add(BatchNormalization())
model.add(TimeDistributed(  MaxPooling2D((3, 3), strides=(2, 2))) )

model.add(TimeDistributed(Conv2D(32, (3,3), kernel_initializer="he_normal", activation='relu')))
model.add(BatchNormalization())
model.add(TimeDistributed(Conv2D(32, (3,3), kernel_initializer="he_normal", activation='relu')))
model.add(BatchNormalization())

model.add(TimeDistributed(  MaxPooling2D((3, 3), strides=(2, 2))) )
model.add(BatchNormalization())

model.add(TimeDistributed(Flatten()))
 
model.add(Dropout(0.8))
model.add(GRU(64, return_sequences=False))
model.add(Dense(num_classes, activation='softmax'))

#write your model here

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [ ]:
optimiser = Adam(lr=0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0) # write the REducelronplateau code here

ES = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto')

callbacks_list = [checkpoint, LR, ES]
#callbacks_list = [LR, ES]


The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
# summarize history for accuracy
fig = plt.figure(figsize=(15,5))
plt.subplot(1, 2, 1)
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

# summarize history for loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()